In [4]:
import glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math

In [5]:
def calc_enmo(some_data):
    # Calculate the magnitue by first squaring all of the x, y, and z value, then summing them, and taking the square root.
    mag = ((some_data.applymap(lambda x : x**2)).sum(axis=1)).transform(lambda x : np.sqrt(x))
    # To calculate ENMO we subtract 1 (Gravity) from the vector magnitudes
    enmo = mag.transform(lambda x : x - 1)
    # Finally if we have any ENMO values less than 0 we round them up.
    enmo.loc[enmo.loc[:] < 0] = 0
    
    return mag, enmo

In [6]:
def calc_mad(some_data, device):
    some_data = some_data.dropna()
    all_mad = {}

    time_name = device + " Time"
    some_data[time_name] = some_data[time_name].apply(lambda x: x.replace(microsecond=0))
        
    # Grab the first timestamp from data
    # print(some_data.shape)
    start = some_data.loc[some_data.index[0], time_name]
    # Specify the amount of time to aggregate over
    agg_len = 5
    # Grab end of aggregation period
    end_time = start + timedelta(seconds=agg_len - 1)
    # Calculate the total length of the trial in seconds
    trial_length = (some_data.loc[some_data.index[-1], time_name] - start).total_seconds()
    # Runs the total length of trial divided by the length of time we aggregate over
    # essentialy creates a window of agg_len, and interval of agg_len
    for i in range(int(trial_length//agg_len)):
        # print(end_time)
        # Get agg_len seconds worth of accelerometer readings
        group_s = some_data.loc[(some_data[time_name] >= start) & (some_data[time_name] <= end_time), :]
        # print(group_s)
        # Get the mean X, Y, and Z of those readings
        agg_s = group_s.aggregate(lambda x : np.mean(x))
        # print(mean_s)
        # Calculate the mean accelerometer magnitude
        mag_s = agg_s[4]
        # print(f"{mag_s}")
        # Subtract the mean magnitude from each accelerometer magnitude from each vector magnitude and then take abs
        dif_mean = group_s[device + ' Magnitude'].apply(lambda x : abs(x - mag_s))
        # Caclulate the sum of all the vector mags - mean mags. Then divide by the number of vectors
        # print(dif_mean.sum())
        if device == "Actigraph":
            if dif_mean.shape[0] != 500 :
                print(f"Error {dif_mean.shape[0]} readings")
        mad = (dif_mean.sum()) / dif_mean.shape[0]
        # print(mad)

        # Add each Mad and the corresponding time to a list :
        all_mad[end_time] = mad
        # 
        start = end_time + timedelta(seconds=1)
        end_time = start + timedelta(seconds=agg_len - 1)

    mad_df = pd.Series(data=all_mad)
    return mad_df

In [12]:
# Define path of file
path = "C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\"
trial = input("Which Trial is being processed? ")
input_path = path + "Trial " + trial + "\\apple_aligned.csv"

Which Trial is being processed? 07


In [13]:
# Creates a dictionary that contains the data of each trial, and its keys are each trial number.
temp = pd.read_csv(input_path)
temp["Actigraph Time"] = pd.to_datetime(temp["Actigraph Time"])
temp["Proxy Time"] = pd.to_datetime(temp["Proxy Time"])
# acti1, acti2, acti3, acti4, acti5 = temp.groupby("Actigraph ID")
acti1, acti2, acti3, acti4 = temp.groupby("Actigraph ID")
# data = [acti1[1], acti2[1], acti3[1], acti4[1], acti5[1]]
data = [acti1[1], acti2[1], acti3[1], acti4[1]]
print("Finished") 

Finished


In [14]:
# Calculate Magnitude and ENMO for each device in each trial.
pair_num = 0
for dev_pair in data :# Each trial has 5 device pairs.
    # Calculate magnitude and ENMO of actigraph
    # print(pair_num)
    acti_mag, acti_enmo = calc_enmo( dev_pair.loc[:, ["Actigraph X", "Actigraph Y", "Actigraph Z"]] )
    dev_pair.insert(10, "Actigraph Magnitude", acti_mag)
    dev_pair.insert(11, "Actigraph ENMO", acti_enmo)
    # Calculate magintude and ENMO of proxy
    proxy_mag, proxy_enmo = calc_enmo( dev_pair.loc[:, ["Proxy X", "Proxy Y", "Proxy Z"]] )
    dev_pair.insert(18, "Proxy Magnitude", proxy_mag)
    dev_pair.insert(19, "Proxy ENMO", proxy_enmo)
    dev_pair.loc[(dev_pair["Proxy X"].isna()), ["Proxy Magnitude", "Proxy ENMO"]] = np.nan
    if pair_num == 0 :
        trial_data = dev_pair
    elif pair_num < 3:
        trial_data = pd.concat([trial_data, dev_pair])
    else :
        trial_data = pd.concat([trial_data, dev_pair])
        trial_data.to_csv(path + "Trial " + trial + "//apple_aligned_v2.csv", index=False)
    pair_num += 1

In [15]:
trial_data

,Trial Number,Round Number,Speed,Actigraph ID,Actigraph Time,Order,Include,Actigraph X,Actigraph Y,Actigraph Z,Actigraph Magnitude,Actigraph ENMO,Proxy ID,Proxy Time,Reading #,Proxy X,Proxy Y,Proxy Z,Proxy Magnitude,Proxy ENMO
504000,7,4,3.2 Hz,20,2022-04-01 14:11:30,504001,0,0.762,-0.348,-0.938,1.257614,0.257614,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
504001,7,4,3.2 Hz,20,2022-04-01 14:11:30,504002,0,0.754,-0.504,-0.949,1.312682,0.312682,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
504002,7,4,3.2 Hz,20,2022-04-01 14:11:30,504003,0,0.605,-0.738,-0.984,1.370739,0.370739,V3KQT9D9NR,2022-04-01 14:11:30.015,NaN,0.017578,0.020264,-0.995361,0.995723,0.0
504003,7,4,3.2 Hz,20,2022-04-01 14:11:30,504004,0,0.379,-0.828,-1.070,1.405036,0.405036,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
504004,7,4,3.2 Hz,20,2022-04-01 14:11:30,504005,0,0.164,-0.707,-1.016,1.248600,0.248600,V3KQT9D9NR,2022-04-01 14:11:30.035,NaN,0.014893,0.019043,-0.994873,0.995167,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335995,7,4,0.6 Hz,84,2022-04-01 14:25:34,335996,0,-0.023,0.004,-1.035,1.035263,0.035263,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
335996,7,4,0.6 Hz,84,2022-04-01 14:25:34,335997,0,-0.023,0.004,-1.035,1.035263,0.035263,GKGGN79P34,2022-04-01 14:25:34.953,NaN,-0.000977,0.023926,-0.995117,0.995405,0.0
335997,7,4,0.6 Hz,84,2022-04-01 14:25:34,335998,0,-0.023,0.004,-1.035,1.035263,0.035263,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
335998,7,4,0.6 Hz,84,2022-04-01 14:25:34,335999,0,-0.023,0.004,-1.035,1.035263,0.035263,GKGGN79P34,2022-04-01 14:25:34.972,NaN,0.005371,0.014404,-0.996582,0.996701,0.0


In [16]:
def aggregate_data(data, device):
    # Calculate the MAD for device
    device_mad = calc_mad(data.loc[:, [device + " Time", device + " X", device + " Y", device + " Z", 
                                               device + " Magnitude"]], device).rename(device + " MAD")
    # Intialize formula used to calculate RMS
    rms = lambda x : np.sqrt(np.mean(x**2))
    # Intialize functions to be used on data columns
    device_aggs = {"Order" : [np.min], device + " X": [rms], device + " Y": [rms], device + " Z": [rms], device + " Magnitude": [np.max],
                    device + " ENMO": [np.max]}
    # Group the actigraph data by second and perform aggregations
    if device == "Proxy":
        data[device + " Time"] = data[device + " Time"].apply(lambda x : x.replace(microsecond=0))
        device_rms = data.drop(columns=["Reading #"]).dropna().groupby(["Include", device + " Time"]).agg(device_aggs)
        # print(device_rms)
    else:
        device_rms = data.groupby(["Include", device + " Time"]).agg(device_aggs)
        # print(device_rms)
                            
    # print(device_rms)
    # Rename aggregate  columns
    device_rms= device_rms.reset_index()
    device_rms.columns = ["Include", device + " Time", "Order", device + " RMS X", device + " RMS Y", device + " RMS Z", 
                        device + " MAX MAGNITUDE", device + " MAX ENMO"]
    # Selecte middle minutes from eacht trial only
    device_rms = device_rms.loc[device_rms["Include"] == 1].drop(columns=["Include"])
    # print(device_rms)
    # Create a data frame that holds the meta trial data
    acti_meta = data.loc[:,["Trial Number", "Round Number", "Speed", device + " ID", "Order"]]
    #if device == "Proxy":
        #print(device_rms)
        #print(device_mad)
    # Merge the MAD calculation with the aggregated x, y, z, magintude, and ENMO
    device_mad = device_rms.merge(device_mad, how="left", left_on=device + " Time", right_on=device_mad.index)
    # Merge aggregated data with trial meta data
    device_sec = acti_meta.merge(device_mad, how='inner', on="Order")
    # print(device_sec.loc[55:65,:])
    return device_sec

In [17]:
# Calculate MAD for each trial, and also aggregate data to second level.
# Used to keep track of which device pair is being looked at
pair_num = 0
# Resets trial file
trial_final = None
# Iterate through each device pair (5 actigraphs  and 2 proxys per trial)
acti_num = 1
for acti_pair in data :
    print(f"Actigraph Number : {acti_num}")
    # acti_pair["Round Number"] = 5 # Used for extra rounds
    # split the data by proxy ID
    
    [proxy_1, proxy_2] = acti_pair.groupby("Proxy ID")
    # if acti_num == 1 :
        # print(f"Proxy 1: {proxy_1[0]} \nProxy 2: {proxy_2[0]}")
        # print(f"Proxy {acti_pair.iloc[1,6]}")
    acti_num += 1
    # print(proxy_1[1])

    # Aggreagte data and calculate mad for actigraph
    acti_data = aggregate_data(acti_pair.iloc[:acti_pair.shape[0]//2, :], "Actigraph")
    # acti_data = aggregate_data(acti_pair, "Actigraph")
    # print(acti_data)

    # Aggreagte data and calculate mad for proxy 1:
    proxy1_data = aggregate_data(proxy_1[1], "Proxy")
    # proxy1_data = aggregate_data(acti_pair, "Proxy")
    # print(proxy1_data)

    # Aggreagte data and calculate mad for proxy 2:
    proxy2_data = aggregate_data(proxy_2[1], "Proxy")

    # Get rid of duplicate columns
    proxy1_data.drop(columns=["Trial Number", 'Round Number', 'Speed', 'Order'], inplace=True)
    # print(proxy1_data)
    proxy2_data.drop(columns=["Trial Number", 'Round Number', 'Speed', 'Order'], inplace=True)
    # combine into one dataframe
    temp_prox1 = acti_data.merge(proxy1_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")

    temp_prox2 = acti_data.merge(proxy2_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")


    if pair_num == 0 :
        trial_final = pd.concat([temp_prox1, temp_prox2])
        # trial_final = acti_data.merge(proxy1_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")
    else:
        temp = pd.concat([temp_prox1, temp_prox2])
        # temp = acti_data.merge(proxy1_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")
        trial_final = pd.concat([trial_final, temp])
        if pair_num == 3 :
            trial_final.to_csv(path + "Trial " + trial + "//apple_rms_v2.csv", index=False)
    pair_num += 1
trial_final

Actigraph Number : 1
Error 0 readings


C:\Users\Nick\AppData\Local\Temp/ipykernel_13064/124379684.py:37: RuntimeWarning: invalid value encountered in double_scalars
  mad = (dif_mean.sum()) / dif_mean.shape[0]


Actigraph Number : 2
Error 0 readings
Actigraph Number : 3
Error 0 readings
Actigraph Number : 4
Error 0 readings


,Trial Number,Round Number,Speed,Actigraph ID,Order,Actigraph Time,Actigraph RMS X,Actigraph RMS Y,Actigraph RMS Z,Actigraph MAX MAGNITUDE,Actigraph MAX ENMO,Actigraph MAD,Proxy ID,Proxy Time,Proxy RMS X,Proxy RMS Y,Proxy RMS Z,Proxy MAX MAGNITUDE,Proxy MAX ENMO,Proxy MAD
0,7,4,3.2 Hz,20,507001,2022-04-01 14:12:00,0.323695,0.361503,1.000157,1.298925,0.298925,NaN,GKGGN79P34,2022-04-01 14:12:00,0.359537,0.330726,0.993553,1.343616,0.343616,NaN
1,7,4,3.2 Hz,20,507101,2022-04-01 14:12:01,0.319598,0.360370,0.998730,1.274056,0.274056,NaN,GKGGN79P34,2022-04-01 14:12:01,0.362812,0.332812,0.996214,1.388644,0.388644,NaN
2,7,4,3.2 Hz,20,507201,2022-04-01 14:12:02,0.329903,0.351154,0.998500,1.276641,0.276641,NaN,GKGGN79P34,2022-04-01 14:12:02,0.380618,0.332891,0.995543,1.381427,0.381427,NaN
3,7,4,3.2 Hz,20,507301,2022-04-01 14:12:03,0.324855,0.354563,0.999041,1.289826,0.289826,NaN,GKGGN79P34,2022-04-01 14:12:03,0.372034,0.324104,0.994860,1.395744,0.395744,NaN
4,7,4,3.2 Hz,20,507401,2022-04-01 14:12:04,0.328230,0.366583,0.998684,1.292869,0.292869,3.659050e-02,GKGGN79P34,2022-04-01 14:12:04,0.383144,0.331634,0.997019,1.382797,0.382797,0.056898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,7,4,0.6 Hz,84,248501,2022-04-01 14:25:00,0.023000,0.004000,1.035000,1.035263,0.035263,NaN,V3KQT9D9NR,2022-04-01 14:25:00,0.012783,0.020904,0.996297,1.011836,0.011836,NaN
416,7,4,0.6 Hz,84,248601,2022-04-01 14:25:01,0.023000,0.004000,1.035000,1.035263,0.035263,NaN,V3KQT9D9NR,2022-04-01 14:25:01,0.022361,0.025515,0.995455,1.012444,0.012444,NaN
417,7,4,0.6 Hz,84,248701,2022-04-01 14:25:02,0.023000,0.004000,1.035000,1.035263,0.035263,NaN,V3KQT9D9NR,2022-04-01 14:25:02,0.013985,0.021155,0.996085,1.012915,0.012915,NaN
418,7,4,0.6 Hz,84,248801,2022-04-01 14:25:03,0.023000,0.004000,1.035000,1.035263,0.035263,NaN,V3KQT9D9NR,2022-04-01 14:25:03,0.022757,0.027132,0.995716,1.014099,0.014099,NaN


In [183]:
# Combine all aligned
aligned_files = glob.glob(path + "Trial *//*aligned_V2.csv")
aligned_stack = None
for file in aligned_files:
    if aligned_stack is None:
        aligned_stack = pd.read_csv(file)
    else:
        temp = pd.read_csv(file)
        aligned_stack = pd.concat([aligned_stack, temp])
aligned_stack.to_csv(path + "//apple_aligned_mad_enmo.csv", index=False)

In [190]:
aggregated_files = glob.glob(path + "Trial *//*rms_v2.csv")
aggregated_stack = None
for file in aggregated_files:
    if aggregated_stack is None:
        aggregated_stack = pd.read_csv(file)
    else:
        temp = pd.read_csv(file)
        aggregated_stack = pd.concat([aggregated_stack, temp])
aggregated_stack.to_csv(path + "//apple_aggregated_mad_enmo.csv")

In [189]:
aggregated_files = glob.glob(path + "Trial *//*rms_v2.csv")
aggregated_files

['C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 01\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 02\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 03\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 04\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 05\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 06\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 06e\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 07\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 08\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 09\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial